In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
from tqdm import tqdm
import time
import os

In [2]:
path = "2d-simu"
if not os.path.exists(path):
    os.makedirs(path)

In [3]:
def create_initial_config(L,N,sigma):
    x = np.zeros((N,2))
    for i in range(N):
        correct_placement = 0
        while(correct_placement == 0):
            x_rand = np.random.rand(2)*L
            correct_placement = 1
            for j in range(i):
                dist = np.linalg.norm(x_rand-x[j,:])
                if dist < 2*sigma:
                    correct_placement = 0
            
            if (x_rand[0] < sigma) or (x_rand[0] > L-sigma):
                correct_placement = 0
                
            if (x_rand[1] < sigma) or (x_rand[1] > L-sigma):
                correct_placement = 0
                    
        x[i,:] = x_rand
    
    v =  np.random.normal(0, 3, (N,2))
    return x, v


#computes the collision time for one particle pair
def compute_tij(x,v,sigma):

    a = np.dot(v,v)
    b = 2*np.dot(v,x)
    c = np.dot(x,x) - (2*sigma)**2
    
    if((b**2-4*a*c)>=0):
        t1 = (-b+np.sqrt(b**2-4*a*c))/(2*a)
        t2 = (-b-np.sqrt(b**2-4*a*c))/(2*a)
        
        if t1 < 0 and t2 > 0:
            return t2
        elif t2 < 0 and t1 > 0:
            return t1
        elif t1 > 0 and t2 > 0:
            if t1 < t2:
                return t1
            else:
                return t2
        else:
            return 10000
        
    else:
        return 10000
    
    
       
#calculate all the collision times for all particles
def get_tij(x,v,N,sigma):

    tij = np.zeros((N,N))
    for i in range(N):
        for j in range(i+1,N):
            xij = x[i,:] - x[j,:]
            vij = v[i,:] - v[j,:]
            tij[i,j] = compute_tij(xij,vij,sigma)
    
    return tij



#also calculate bounce of times with the walls:
def get_boundary_collision(x,v,N,L,sigma):
    tr = np.zeros(N)
    for i in range(N):
        bi = np.zeros(2)
        
        if v[i,0] < 0:
            bi[0] = (x[i,0]-sigma)/np.abs(v[i,0])
        elif v[i,0] > 0:
            bi[0] = (L-x[i,0]-sigma)/np.abs(v[i,0])
        else:
            b[i,0] = 1000
            
        if v[i,1] < 0:
            bi[1] = (x[i,1]-sigma)/np.abs(v[i,1])
        elif v[i,1] > 0:
            bi[1] = (L-x[i,1]-sigma)/np.abs(v[i,1])
        
        else:
            b[i,1] = 1000
            
        tr[i] = min(bi)
    return tr

## Initialization

In [4]:
#create inital configuration
L = 5 # length of 1d line
N = 12 # number of particles
sigma = 0.1 # radius of particle
x, v = create_initial_config(L,N,sigma)
end_time = 6
dt = 0.01
steps = int(end_time/dt)
epsilon = 0.1 #margin for checing if the particle hits the boundary

## Simulation

In [5]:
counter = 0

while (counter < steps):
    #calculate next collision with wall or particle
    
    #first find the next particle collisions
    tij = get_tij(x,v,N,sigma) 
    tc = np.min(tij[np.nonzero(tij)]) # minimum time of next collision
    idx = np.where(tij == tc) # indices of the two particle that colide next
    i = idx[0][0]
    j = idx[1][0]
    
    #second find the next collision with the boundary
    t_boundary = get_boundary_collision(x,v,N,L,sigma)
    tb = np.min(t_boundary) # time of next boundary collision
    k = np.where(t_boundary == tb)[0][0] # index of the particle that collides with the boundary
    
    if tc < tb:
        # next collision is between particles
        for timer in range(int(tc/dt)):
            x = x+v*dt
            
            counter += 1
            #save the plot
            name = str(counter)+'.png'
            fig = plt.figure(figsize=(10, 10))
            plt.yticks([])
            plt.xticks([])
            plt.ylim([0,L])
            plt.xlim([0,L])
            for p in range(N):
                plt.scatter(x[p,0],x[p,1],s=600)
            plt.savefig(os.path.join(path, name) ,dpi=200)
            plt.close(fig)
            
        #collision
        xij = x[i,:]-x[j,:]
        vij = v[i,:]-v[j,:]
        xji = -xij
        vji = -vij

        v[i,:] = v[i,:] - (xij)*(np.dot(vij,xij))/(np.dot(xij,xij))
        v[j,:] = v[j,:] - (xji)*(np.dot(vji,xji))/(np.dot(xji,xji))
        
            
    else:
        # net collision is with the boundary
        for timer in range(int(tb/dt)):
            x = x+v*dt
        
            counter += 1
            #save the plot
            name = str(counter)+'.png'
            fig = plt.figure(figsize=(10, 10))
            plt.yticks([])
            plt.xticks([])
            plt.ylim([0,L])
            plt.xlim([0,L])
            for p in range(N):
                plt.scatter(x[p,0],x[p,1],s=600)
            plt.savefig(os.path.join(path, name), dpi=200)
            plt.close(fig)
        
        #collision with the boundary
        if (x[k,0] > L-sigma-epsilon) or (x[k,0] < sigma+epsilon):
            v[k,0] = -v[k,0]
        else:
            v[k,1] = -v[k,1]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
